# CS 425 fall 2021 Assignment 4


## Multiple sequence alignment


Implement the progressive multiple sequence alignment strategy described in class which uses the sum-of-pairs score for pairwise alignment:

$$
s(m_i) = \sum_{j < k} \delta(m_{ij}, m_{jk}),
$$
where $m_i$ is column $i$ of a multiple sequence alignment and $\delta(\cdot, \cdot) is the scoring matrix.  Note that the sum-of-pairs score can also be used to score the similarity of two columns when aligning two alignments during progressive alignment.
In your code, use the [BLOSUM 62 substitution matrix](http://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt) as the scoring matrix.

For ordering the sequences for alignment use the shared k-mers normalized similarity measure we looked at in notebook 6.
The non-normalized version is:

$$\mathrm{sim}(s_1, s_2, k) = \sum_{\mathrm{kmer \in s_1}} \min(\mathrm{counts}(kmer, s_1), \mathrm{counts}(kmer, s_2) ),$$

where $s_1,s_2$ are two protein sequences, $k$ is the kmer length and $\mathrm{counts}(kmer, s)$ is the number of times the given kmer appears in the sequence $s$.
This measure of similarity is then normalized to be between 0 and 1:

$$\mathrm{sim}_{norm}(s_1, s_2, k) = \frac{\mathrm{sim}(s_1, s_2, k)}{\max( \mathrm{sim}(s_1, s_1, k), \mathrm{sim}(s_2, s_2, k) )}$$

In your experiments, use $k=3$.

For ordering and aligning the sequences use the following simple greedy method:

* Find the two most similar sequences and align them.
* Among the remaining sequences, find the one that is closest to the sequences in the existing alignment (use the "single linkage" way of measuring similarity to a set of sequences).  As the similarity, use the k-mer based similarity defined above.
Align that sequence to the existing alignment.  Continue until all sequences have been aligned.

This is a bit simpler than the progressive alignment method presented in class.

To develop your code use the following short sequences:

In [234]:
sequence_dict = {
    'Guinea pig': 'FSIVPKTPLQISGIEEDSDDPVERRL',
    'Zebrafish': 'ASFIQVPEEEVRRTLPDRKF',
    'Human': 'FSIVQKTPLQMNGIEEDSDEPLERRL',
    'Mouse': 'ISIVQKTPLCIDGESDDLQEKRL',
    'Small-eared galago': 'FSVVQKTPLPMNGIDEEDSEEPVERRL',
    'Rat': 'ISIVQKTPLSIEGESDDLQERRL',
    'Spiny dogfish': 'FSLVQTAMSYPQTNGMEDATSEPGERHF'
}

sequence_list = ['FSIVPKTPLQISGIEEDSDDPVERRL', 'ASFIQVPEEEVRRTLPDRKF', 'FSIVQKTPLQMNGIEEDSDEPLERRL', 'ISIVQKTPLCIDGESDDLQEKRL', 'FSVVQKTPLPMNGIDEEDSEEPVERRL', 'ISIVQKTPLSIEGESDDLQERRL', 'FSLVQTAMSYPQTNGMEDATSEPGERHF']

For final development, use the set of [globin sequences](https://raw.githubusercontent.com/asabenhur/CS425/main/data/globins.fasta) we have previously analyzed.

In [235]:
import numpy as np
import collections
from matplotlib import pyplot as plt

In [236]:
DIAG = 0 
UP = 1 
LEFT = 2
STORE=[]
longest_length = max(len(s) for s in sequence_list)

arrows = [u"\u2196", u"\u2191", u"\u2190"]

def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, indel=-1, verbose=True) :
    # s,ptr = needleman_wunsch_matrix(seq1, seq2, match, mismatch, indel)
    # alignment = needleman_wunsch_trace(seq1, seq2, s, ptr)

    if verbose :
        s,ptr = needleman_wunsch_matrix(seq1, seq2, match, mismatch, indel)
        alignment = needleman_wunsch_trace(seq1, seq2, s, ptr)
        print('\n'+'~`'*25)
        # print("Alignment Score: %f\n" % (s[len(seq1),len(seq2)]))
        print("Alignment:")
        print(alignment[0])
        print(alignment[1])
        STORE.append(alignment[0])
        STORE.append(alignment[1])

    if verbose == False:

        s,ptr = needleman_wunsch_matrix(STORE[0], seq2, match, mismatch, indel)
        alignment = needleman_wunsch_trace(STORE[0], seq2, s, ptr)

        print(alignment[0])
        # STORE.append(alignment[1])
        
    
    return alignment
    



def needleman_wunsch_matrix(seq1, seq2, match=1, mismatch=-1, indel=-1):

    n = len(seq1)
    m = len(seq2)
    s = np.zeros( (n+1, m+1) ) # DP matrix
    ptr = np.zeros( (n+1, m+1), dtype=int  ) # matrix of pointers

    ##### INITIALIZE SCORING MATRIX (base case) #####

    for i in range(1, n+1) :
        s[i,0] = indel * i
    for j in range(1, m+1):
        s[0,j] = indel * j
        
    ########## INITIALIZE TRACEBACK MATRIX ##########

    # Tag first row by LEFT, indicating initial '-'s
    ptr[0,1:] = LEFT
        
    # Tag first column by UP, indicating initial '-'s
    ptr[1:,0] = UP

    #####################################################

    for i in range(1,n+1):
        for j in range(1,m+1): 
            # match
            if seq1[i-1] == seq2[j-1]:
                s[i,j] = s[i-1,j-1] + match
                ptr[i,j] = DIAG
            # mismatch
            else :
                s[i,j] = s[i-1,j-1] + mismatch
                ptr[i,j] = DIAG
            # indel penalty
            if s[i-1,j] + indel > s[i,j] :
                s[i,j] = s[i-1,j] + indel
                ptr[i,j] = UP
            # indel penalty
            if s[i, j-1] + indel > s[i,j]:
                s[i,j] = s[i, j-1] + indel
                ptr[i,j] = LEFT

    return s, ptr

def needleman_wunsch_trace(seq1, seq2, s, ptr) :

    #### TRACE BEST PATH TO GET ALIGNMENT ####
    align1 = ""
    align2 = ""
    n, m = (len(seq1), len(seq2))
    i = n
    j = m
    curr = ptr[i, j]
    while (i > 0 or j > 0):        
        ptr[i,j] += 3
        if curr == DIAG :            
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1            
        elif curr == LEFT:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1            
        elif curr == UP:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
           
        curr = ptr[i,j]

    return align1, align2

In [237]:
def counts(k, seq):
  store = collections.defaultdict(int)

  for start in range(3):
    for i in range(start,len(seq),k):
      if i+k < len(seq):
        store[seq[i:i+k]] += 1

  return store

def kmer_similarity(seq1, seq2, k=3) :
    x,y = counts(k,seq1),counts(k,seq2)
    dd = collections.defaultdict(list)

    count = 0

    # THIS COMBINES BOTH DICT'S INTO 1
    # IF BOTH DICTIONAIRES HAVE SAME KEY'S THEN THE VALUES ARE PUT INTO A LIST
    for d in (x,y):
      for key, val in d.items():
        dd[key].append(val)

    # FINDS SIMILARITES OF BOTH SEQUENCES
    for val in dd.items():
      if len(val[1]) > 1:
        count += min(val[1])

    return count
  
def similarities(sequences):
  matrix = np.zeros((len(sequences), len(sequences)),np.float)

  for i, seq1 in enumerate(sequences):
    for j, seq2 in enumerate(sequences):
      if i != j:
        numerator =  kmer_similarity(sequences[i], sequences[j])
        denomerator1 = kmer_similarity(sequences[i],sequences[i])
        denomerator2 = kmer_similarity(sequences[j], sequences[j])
        final = numerator/max(denomerator1, denomerator2)

        matrix[i][j] = final
  
  return matrix

def findHighestScore(matrix):
  maxScore = 0

  for i in range(len(matrix)):
    for j in range(len(matrix[i])):
      if matrix[i][j] > maxScore:
        maxScore = matrix[i][j]
        location = [i,j]

  return location


In [238]:
#READ FILE
def read_fasta(file_name):
  data = []

  with open(file_name,"r") as f:
    for count, line in enumerate(f, start=1):
        if count % 2 == 0:
          data.append(line[:len(line)-3])

  return data

if __name__ == "__main__":
  sequence_list = read_fasta("globins.fasta")
  # print(len(sequence_list))

  scoring_matrix = similarities(sequence_list)
  # print(scoring_matrix)

  location = findHighestScore(scoring_matrix)

  needleman_wunsch(sequence_list[location[0]], sequence_list[location[1]])
  scoring_matrix[location[0],location[1]] = 0
  scoring_matrix[location[1],location[0]] = 0

  same = False

  
  # padded_sequences = [s.ljust(longest_length, '-') for s in sequence_list]
  while(same == False):
    location = findHighestScore(scoring_matrix)
    # loc=closet(scoring_matrix)

    needleman_wunsch(sequence_list[location[0]], sequence_list[location[0]],verbose=False)
    needleman_wunsch(sequence_list[location[1]], sequence_list[location[1]],verbose=False)
    scoring_matrix[location[0],location[1]] = 0
    scoring_matrix[location[1],location[0]] = 0
    # print(scoring_matrix)
    same = np.all(scoring_matrix==0)

  # for x in STORE:
  #   print(x)


  


~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`
Alignment:
-------------eelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsqfkhmedplemerspqlrkhacrvmgalntvvenlhdpdkvssvlalvgkahalkhkvepvyfkilsgvilevvaeefasdfppetqrawaklrgliyshvtaaykevgwvqqvpnattppatlp
vpgemeierrerseelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsqfkhmedplemerspqlrkhacrvmgalntvvenlhdpdkvssvlalvgkahalkhkvepvyfkilsgvilevvaeefasdfppetqrawaklrgliyshvtaayke------v-----------
-------------eelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsqfkhmedplemerspqlrkhacrvmgalntvvenlhdpdkvssvlalvgkahalkhkvepvyfkilsgvilevvaeefasdfppetqrawaklrgliyshvtaaykevgwvqqvpnattppatlp
-------------eelseaerkavqamwarlyancedvgvailvrffvnfpsakqyfsqfkhmedplemerspqlrkhacrvmgalntvvenlhdpdkvssvlalvgkahalkhkvepvyfkilsgvilevvaeefasdfppetqrawaklrgliyshvtaaykevgwvqqvpnattppatlp
-------------eelseaerkavqa-mw-arlya-ncedvgvailvrffvnfp-sakqyfsqfkhmedplemerspqlrkhacrvmgalntv-venlhdpdk-vssvlalvg---kahalkh-kvepvyfkilsgvilevvaeefasdfppetqrawaklrgliyshvtaaykevgwvqqvpnattppatlp
-------

KeyboardInterrupt: ignored

Run your method on the development data and the globin sequence data.  Inspect the resulting alignment and discuss the results. To visualize the alignments it is recommended you use the [MView MSA viewer](https://www.ebi.ac.uk/Tools/msa/mview/). Where do you think it could have done better?
Compare the alignments you got in terms of sum-of-pairs score to the result you get using [Muscle](https://www.ebi.ac.uk/Tools/msa/muscle/).  

### Extra credit 1:  Iterative refinement

Modify your code to perform a round of iterative refinement:
Iterate over all sequences in the alignment.  At each step remove a sequence from the alignment.  Re-align the sequence to the alignment and accept the new alignment if it improves the overall score of the alignment.
Compare the sum-of-pairs score before and after the iterative refinement step.

### Extra credit 2:  using a guide tree

Modify your code to perform progressive alignment using a guide tree computed e.g. using agglomerative clustering using scikit-learn.  Compare the sum-of-pairs score of the two methods.  Which one is performing better?

### Submission

Submit your assignment as a Jupyter notebook via Canvas.  

### Grading 

Here is what the grading sheet will look like for this assignment. 

```
Grading sheet for assignment 4

Implementation of progressive alignment method (90 pts)
Discussion of results (10 pts)
Extra credit 1 (10 pts)
Extra credit 2 (10 pts)

```
